# Capstone 2: COVID-19
## Step 01: Data Wrangling

In this notebook the available data is reviewed and processed to prepare it for the later data analysis steps. All raw data is found in the subfolder data/raw/. Individual sources and characteristics will be given below per data file or source.

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score,explained_variance_score,mean_absolute_error
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

%matplotlib inline
os.getcwd()

'/Users/lisahw/Documents/Courses and Conferences/DataScience/MyProject/Capstone_02/Springboard/notebooks'

In [2]:
raw_data = '../data/raw/'
int_data = '../data/interim/'
date_covid = '05-09-2020'

### COVID-19 data 

In [198]:
covid_link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(date_covid)
df_covid = pd.read_csv(covid_link)
df_covid.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-05-10 02:32:30,34.223334,-82.461707,34,0,0,34,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-05-10 02:32:30,30.295065,-92.414197,151,11,0,140,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-05-10 02:32:30,37.767072,-75.632346,496,7,0,489,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-05-10 02:32:30,43.452658,-116.241552,731,19,0,712,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-05-10 02:32:30,41.330756,-94.471059,3,0,0,3,"Adair, Iowa, US"


In [199]:
df_covid.drop(df_covid.index[df_covid.Combined_Key.str.contains('Recovered')],inplace=True)
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3230 entries, 0 to 3231
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FIPS            2957 non-null   float64
 1   Admin2          2960 non-null   object 
 2   Province_State  3047 non-null   object 
 3   Country_Region  3230 non-null   object 
 4   Last_Update     3230 non-null   object 
 5   Lat             3165 non-null   float64
 6   Long_           3165 non-null   float64
 7   Confirmed       3230 non-null   int64  
 8   Deaths          3230 non-null   int64  
 9   Recovered       3230 non-null   int64  
 10  Active          3230 non-null   int64  
 11  Combined_Key    3230 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 328.0+ KB


In [5]:
len(df_covid.Country_Region.unique())

187

The above tells us that we have information on 187 different countries.

Essential information to be retireved from df_covid:
Country/State, Confirmed, Deaths, Recovered, Active
Some countries are split into different regions. Such detailed information is not avaibale (for now) in the other data sets. Therefore regional information is summed up to national information below. This can be revisited again.


In [200]:
first = True
for country in df_covid.Country_Region.unique():
    bool_c = df_covid.Country_Region==country
    if len(bool_c) > 0:
        print(country)
        if first:
            first = False
            dic = {'Country':[country],'Confirmed': [sum(df_covid.Confirmed[bool_c])],'Deaths': [sum(df_covid.Deaths[bool_c])],'Recovered': [sum(df_covid.Recovered[bool_c])],'Active': [sum(df_covid.Active[bool_c])]}
            df_base = pd.DataFrame(dic)
        else:
            dic = {'Country':country,'Confirmed': sum(df_covid.Confirmed[bool_c]),'Deaths': sum(df_covid.Deaths[bool_c]),'Recovered': sum(df_covid.Recovered[bool_c]),'Active': sum(df_covid.Active[bool_c])}
            df_base = df_base.append(dic,ignore_index=True)        

US
Canada
United Kingdom
China
Netherlands
Australia
Denmark
France
Afghanistan
Albania
Algeria
Andorra
Angola
Antigua and Barbuda
Argentina
Armenia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burma
Burundi
Cabo Verde
Cambodia
Cameroon
Central African Republic
Chad
Chile
Colombia
Comoros
Congo (Brazzaville)
Congo (Kinshasa)
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Cyprus
Czechia
Diamond Princess
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Fiji
Finland
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Grenada
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Holy See
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Korea, South
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Liberia
Libya
Liechtenstein
Lithuania
Luxembourg
MS Zaandam
Madagascar
Malawi
Mala

In [201]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    187 non-null    object
 1   Confirmed  187 non-null    int64 
 2   Deaths     187 non-null    int64 
 3   Recovered  187 non-null    int64 
 4   Active     187 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.4+ KB


### Cardiovascular Death Rate

In [8]:
cardio_file = raw_data + 'cardiovascular-death-rate-vs-gdp-per-capita.csv'
# Source: https://ourworldindata.org/grapher/cardiovascular-death-rate-vs-gdp-per-capita
df_cardio = pd.read_csv(cardio_file)
df_cardio.head()

,Entity,Code,Year,"Cardiovascular disease death rate (per 100,000) (Rate)","GDP per capita, PPP (constant 2011 international $) (constant 2011 international $)",Total population (Gapminder)
0,Afghanistan,AFG,1800,NaN,NaN,3280000.0
1,Afghanistan,AFG,1820,NaN,NaN,3280000.0
2,Afghanistan,AFG,1870,NaN,NaN,4207000.0
3,Afghanistan,AFG,1913,NaN,NaN,5730000.0
4,Afghanistan,AFG,1950,NaN,NaN,8151455.0


In [151]:
df_cardio[df_cardio.Entity=='Afghanistan']

,Entity,Code,Year,"Cardiovascular disease death rate (per 100,000) (Rate)","GDP per capita, PPP (constant 2011 international $) (constant 2011 international $)",Total population (Gapminder)
0,Afghanistan,AFG,1800,NaN,NaN,3280000.0
1,Afghanistan,AFG,1820,NaN,NaN,3280000.0
2,Afghanistan,AFG,1870,NaN,NaN,4207000.0
3,Afghanistan,AFG,1913,NaN,NaN,5730000.0
4,Afghanistan,AFG,1950,NaN,NaN,8151455.0
...,...,...,...,...,...,...
67,Afghanistan,AFG,2013,626.162149,1848.700026,34499915.0
68,Afghanistan,AFG,2014,616.669379,1838.960244,NaN
69,Afghanistan,AFG,2015,608.301518,1809.016488,NaN
70,Afghanistan,AFG,2016,601.511737,1802.695565,NaN


In [9]:
print(sum(df_cardio.Entity=='United States'))
df_cardio.Entity[df_cardio.Entity.str.contains('United States')].unique()

200


array(['United States', 'United States Virgin Islands'], dtype=object)

In [10]:
df_cardio.loc[df_cardio.index[df_cardio.Entity=='United States'],'Entity'] = 'US'

In [11]:
df_cardio.rename(columns={df_cardio.columns[3]:'Cardio Death Rate'},inplace=True)
df_cardio = df_cardio.loc[:,['Entity','Year','Cardio Death Rate']]
df_cardio.head()


,Entity,Year,Cardio Death Rate
0,Afghanistan,1800,NaN
1,Afghanistan,1820,NaN
2,Afghanistan,1870,NaN
3,Afghanistan,1913,NaN
4,Afghanistan,1950,NaN


In [12]:
def extract_recent(df):
    df.dropna()
    return [list(df['Cardio Death Rate'].loc[df.Year == df.Year.max()])[0],df.Year.max()]
cardio = []
for country in df_cardio.Entity.unique():
    cardio.append(np.hstack((country,extract_recent(df_cardio[df_cardio.Entity==country]))))


In [13]:
df_cardio_2 = pd.DataFrame(cardio)
df_cardio_2.drop(2,axis=1,inplace=True)
df_cardio_2.rename(columns={0:'Country',1:'Cardio Death Rate'},inplace=True)
df_cardio_2.head()


,Country,Cardio Death Rate
0,Afghanistan,597.0293320369332
1,Albania,304.1954064017878
2,Algeria,278.36387712753714
3,American Samoa,283.7503666306692
4,Andean Latin America,116.85012926340202


In [14]:
df_cardio_2['Cardio Death Rate'] = pd.to_numeric(df_cardio_2['Cardio Death Rate'],errors='coerce')
df_cardio_2['Cardio Death Rate'] = df_cardio_2['Cardio Death Rate']/1000

#### Merge datasets

In [15]:
# Only demographic data
df_demo = df_cardio_2

### Diabetes

In [18]:
diabetes_file = raw_data + 'diabetes-prevalence.csv'
# Source: https://ourworldindata.org/grapher/diabetes-prevalence
df_diab = pd.read_csv(diabetes_file)
df_diab.head()

,Entity,Code,Year,Diabetes prevalence (% of population ages 20 to 79) (% of population ages 20 to 79)
0,Afghanistan,AFG,2017,9.59
1,Albania,ALB,2017,10.08
2,Algeria,DZA,2017,6.73
3,Andorra,AND,2017,7.97
4,Angola,AGO,2017,3.94


In [193]:
df_diab.Year.unique()

array([2017])

In [19]:
df_diab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 4 columns):
 #   Column                                                                               Non-Null Count  Dtype  
---  ------                                                                               --------------  -----  
 0   Entity                                                                               253 non-null    object 
 1   Code                                                                                 207 non-null    object 
 2   Year                                                                                 253 non-null    int64  
 3   Diabetes prevalence (% of population ages 20 to 79) (% of population ages 20 to 79)  253 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 8.0+ KB


In [20]:
df_diab.loc[df_diab.index[df_diab.Entity=='United States'],'Entity'] = 'US'

#### Merge datasets

In [22]:
df_diab.rename(columns={'Entity':'Country',df_diab.columns[-1]:'Diabetes Percentage'},inplace = True)
df_demo = pd.merge(left=df_demo,right=df_diab.loc[:,['Country','Diabetes Percentage']],how='outer', on='Country')
df_demo.head()

,Country,Cardio Death Rate,Diabetes Percentage
0,Afghanistan,0.597029,9.59
1,Albania,0.304195,10.08
2,Algeria,0.278364,6.73
3,American Samoa,0.283750,NaN
4,Andean Latin America,0.116850,NaN


### Obesity

In [23]:
obesity_file = raw_data + 'Food_Supply_Quantity_kg_Data.csv'
# Source: https://www.kaggle.com/mariaren/covid19-healthy-diet-dataset
df_obe = pd.read_csv(obesity_file)
df_obe.head()

,Country,Alcoholic Beverages,Animal fats,Animal Products,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Vegetables,Vegetal Products,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population,Unit (all except Population)
0,Afghanistan,0.0014,0.1973,9.4341,0.0,24.8097,0.2099,0.0350,5.3495,1.2020,...,6.7642,40.5645,4.5,29.8,0.006490,0.000189,0.000870,0.005431,38042000.0,%
1,Albania,1.6719,0.1357,18.7684,0.0,5.7817,0.5815,0.2126,6.7861,1.8845,...,11.7753,31.2304,22.3,6.2,0.027607,0.001085,0.018160,0.008362,2858000.0,%
2,Algeria,0.2711,0.0282,9.6334,0.0,13.6816,0.5277,0.2416,6.3801,1.1305,...,11.6484,40.3651,26.6,3.9,0.009895,0.001057,0.004313,0.004525,43406000.0,%
3,Angola,5.8087,0.0560,4.9278,0.0,9.1085,0.0587,1.7707,6.0005,2.0571,...,2.3041,45.0722,6.8,25,0.000111,0.000006,0.000035,0.000070,31427000.0,%
4,Antigua and Barbuda,3.5764,0.0087,16.6613,0.0,5.9960,0.2274,4.1489,10.7451,5.6888,...,5.4495,33.3233,19.1,NaN,0.025773,0.003093,0.015464,0.007216,97000.0,%


In [24]:
df_obe.Undernourished.loc[df_obe.index[df_obe['Undernourished']=='<2.5']] = 1
df_obe.Undernourished = pd.to_numeric(df_obe.Undernourished,errors='coerce')

/Users/lisahw/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [27]:
df_obe.Country.loc[df_obe.Country.str.contains('United States')] = 'US'

#### Merge datasets

In [29]:
df_demo = pd.merge(left=df_demo, right=df_obe.loc[:,['Country','Obesity','Undernourished']],how='outer',on='Country')
df_demo.head()

,Country,Cardio Death Rate,Diabetes Percentage,Obesity,Undernourished
0,Afghanistan,0.597029,9.59,4.5,29.8
1,Albania,0.304195,10.08,22.3,6.2
2,Algeria,0.278364,6.73,26.6,3.9
3,American Samoa,0.283750,NaN,NaN,NaN
4,Andean Latin America,0.116850,NaN,NaN,NaN


### Age Groups

In [30]:
age_file = raw_data + 'WPP2019_PopulationByAgeSex_Medium.csv'
# Source: https://population.un.org/wpp/Download/Standard/CSV/
df_age = pd.read_csv(age_file)
df_age.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,AgeGrp,AgeGrpStart,AgeGrpSpan,PopMale,PopFemale,PopTotal
0,4,Afghanistan,2,Medium,1950,1950.5,0-4,0,5,630.044,661.578,1291.622
1,4,Afghanistan,2,Medium,1950,1950.5,5-9,5,5,516.206,487.335,1003.541
2,4,Afghanistan,2,Medium,1950,1950.5,10-14,10,5,461.378,423.326,884.704
3,4,Afghanistan,2,Medium,1950,1950.5,15-19,15,5,414.369,369.363,783.732
4,4,Afghanistan,2,Medium,1950,1950.5,20-24,20,5,374.109,318.392,692.501


In [79]:
df_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404753 entries, 0 to 1404752
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   LocID        1404753 non-null  int64  
 1   Location     1404753 non-null  object 
 2   VarID        1404753 non-null  int64  
 3   Variant      1404753 non-null  object 
 4   Time         1404753 non-null  int64  
 5   MidPeriod    1404753 non-null  float64
 6   AgeGrp       1404753 non-null  object 
 7   AgeGrpStart  1404753 non-null  int64  
 8   AgeGrpSpan   1404753 non-null  int64  
 9   PopMale      1404753 non-null  float64
 10  PopFemale    1404753 non-null  float64
 11  PopTotal     1404753 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 128.6+ MB


In [198]:
df_age[(df_age.Location=='Afghanistan')&(df_age.Time==2019)]

,LocID,Location,VarID,Variant,Time,MidPeriod,AgeGrp,AgeGrpStart,AgeGrpSpan,PopMale,PopFemale,PopTotal
1449,4,Afghanistan,2,Medium,2019,2019.5,0-4,0,5,2891.330,2747.452,5638.782
1450,4,Afghanistan,2,Medium,2019,2019.5,5-9,5,5,2765.393,2636.519,5401.912
1451,4,Afghanistan,2,Medium,2019,2019.5,10-14,10,5,2614.937,2501.560,5116.497
1452,4,Afghanistan,2,Medium,2019,2019.5,15-19,15,5,2321.520,2197.654,4519.174
1453,4,Afghanistan,2,Medium,2019,2019.5,20-24,20,5,1950.650,1843.985,3794.635
1454,4,Afghanistan,2,Medium,2019,2019.5,25-29,25,5,1551.332,1433.056,2984.388
1455,4,Afghanistan,2,Medium,2019,2019.5,30-34,30,5,1255.855,1138.037,2393.892
1456,4,Afghanistan,2,Medium,2019,2019.5,35-39,35,5,1033.269,954.327,1987.596
1457,4,Afghanistan,2,Medium,2019,2019.5,40-44,40,5,834.402,758.533,1592.935
1458,4,Afghanistan,2,Medium,2019,2019.5,45-49,45,5,649.695,603.870,1253.565


In [31]:
def cluster_agegroup(df):
    bool_age = (df.AgeGrp=='75-79')|(df.AgeGrp=='80-84')|(df.AgeGrp=='85-89')|(df.AgeGrp=='90-94')|(df.AgeGrp=='95-99')|(df.AgeGrp=='100+')
    df2 = df.loc[:,['PopMale','PopFemale','PopTotal']]
    age75plus = df2[bool_age].agg(sum)/df2.PopTotal.agg(sum)*100
    age75plus = age75plus.append(pd.Series({'Total Population':df2.PopTotal.agg(sum)}))
    return age75plus

In [44]:
df_t = pd.DataFrame()
for country in df_age.Location.unique():
    df_tt = cluster_agegroup(df_age[(df_age.Location==country)&(df_age.Time==2019)])
    df_tt['Country'] = country
    df_tt = pd.DataFrame(df_tt).transpose()
    df_t=pd.concat([df_t,df_tt])
df_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 0 to 0
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   PopMale           440 non-null    object
 1   PopFemale         440 non-null    object
 2   PopTotal          440 non-null    object
 3   Total Population  440 non-null    object
 4   Country           440 non-null    object
dtypes: object(5)
memory usage: 20.6+ KB


In [61]:
df_t.PopMale = df_t.PopMale.astype(float)
df_t.PopFemale = df_t.PopFemale.astype(float)
df_t.PopTotal = df_t.PopTotal.astype(float)
df_t['Total Population'] = df_t['Total Population'].astype(float)

In [51]:
df_t.reset_index(inplace=True)

In [206]:
df_t.at[df_t.index[df_t.Country=='United States of America'],'Country']='US'

In [56]:
df_t.drop('index',axis=1,inplace=True)
df_t.head()

,PopMale,PopFemale,PopTotal,Total Population,Country
0,0.313169,0.380924,0.694093,38041.8,Afghanistan
1,0.436261,0.608594,1.04486,1.30806e+06,Africa
2,0.435197,0.607064,1.04226,1.30632e+06,African Group
3,0.435167,0.606961,1.04213,1.3069e+06,African Union
4,0.353841,0.487416,0.841258,154014,African Union: Central Africa


#### Merge datasets

In [58]:
df_demo = pd.merge(left=df_demo,right=df_t,how='outer',on='Country')
df_demo.head()

,Country,Cardio Death Rate,Diabetes Percentage,Obesity,Undernourished,PopMale,PopFemale,PopTotal,Total Population
0,Afghanistan,0.597029,9.59,4.5,29.8,0.313169,0.380924,0.694093,38041.8
1,Albania,0.304195,10.08,22.3,6.2,2.71733,3.06084,5.77817,2880.91
2,Algeria,0.278364,6.73,26.6,3.9,1.15002,1.3069,2.45692,43053.1
3,American Samoa,0.283750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andean Latin America,0.116850,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df_demo.PopMale = df_demo.PopMale.astype(float)
df_demo.PopFemale = df_demo.PopFemale.astype(float)
df_demo.PopTotal = df_demo.PopTotal.astype(float)
df_demo['Total Population'] = df_demo['Total Population'].astype(float)

In [65]:
df_demo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 574 entries, 0 to 573
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              574 non-null    object 
 1   Cardio Death Rate    231 non-null    float64
 2   Diabetes Percentage  253 non-null    float64
 3   Obesity              167 non-null    float64
 4   Undernourished       163 non-null    float64
 5   PopMale              440 non-null    float64
 6   PopFemale            440 non-null    float64
 7   PopTotal             440 non-null    float64
 8   Total Population     440 non-null    float64
dtypes: float64(8), object(1)
memory usage: 44.8+ KB


### Identify Country Definitions that are not needed or coherent

In [83]:
bool_nan = np.isnan(df_demo['Cardio Death Rate']) | np.isnan(df_demo['Diabetes Percentage']) | np.isnan(df_demo['Undernourished'])|np.isnan(df_demo['PopMale'])
np.sort(df_demo.Country[bool_nan].unique())

array(['Africa', 'African Group', 'African Union',
       'African Union: Central Africa', 'African Union: Eastern Africa',
       'African Union: Northern Africa', 'African Union: Southern Africa',
       'African Union: Western Africa',
       'African, Caribbean and Pacific (ACP) Group of States',
       'American Samoa', 'Andean Community', 'Andean Latin America',
       'Andorra', 'Anguilla', 'Antigua and Barbuda', 'Arab World',
       'Aruba', 'Asia', 'Asia-Pacific Economic Cooperation (APEC)',
       'Asia-Pacific Group',
       'Association of Southeast Asian Nations (ASEAN)', 'Australasia',
       'Australia/New Zealand', 'BRIC', 'BRICS', 'Bahamas', 'Bahrain',
       'Belt-Road Initiative (BRI)', 'Belt-Road Initiative: Africa',
       'Belt-Road Initiative: Asia', 'Belt-Road Initiative: Europe',
       'Belt-Road Initiative: Latin America and the Caribbean',
       'Belt-Road Initiative: Pacific', 'Bermuda', 'Bhutan',
       'Black Sea Economic Cooperation (BSEC)',
       'Bol

In [189]:
# Drop political/economical groups or organisations
countries_drop =['African Group', 'African Union','African Union: Central Africa', 'African Union: Eastern Africa',       'African Union: Northern Africa', 'African Union: Southern Africa',       'African Union: Western Africa',       'African, Caribbean and Pacific (ACP) Group of States','Andean Community','Andean Latin America','Arab World','Asia-Pacific Economic Cooperation (APEC)',       'Asia-Pacific Group',    'Association of Southeast Asian Nations (ASEAN)', 'Australasia',       'Australia/New Zealand', 'BRIC', 'BRICS','Belt-Road Initiative (BRI)', 'Belt-Road Initiative: Africa',       'Belt-Road Initiative: Asia', 'Belt-Road Initiative: Europe',       'Belt-Road Initiative: Latin America and the Caribbean',       'Belt-Road Initiative: Pacific','Black Sea Economic Cooperation (BSEC)',       'Bolivarian Alliance for the Americas (ALBA)','Bolivia (Plurinational State of)','Caribbean',
       'Caribbean Community and Common Market (CARICOM)',
       'Caribbean small states', 'Central America','England','United Kingdom (and dependencies)','United States of America (and dependencies)',
       'Central Asia', 'Central Europe', 'Central Europe and the Baltics',
       'Central Europe, Eastern Europe, and Central Asia',       'Central European Free Trade Agreement (CEFTA)',       'Central Latin America', 'Central Sub-Saharan Africa',       'Central and Southern Asia', 'Channel Islands',       'China (and dependencies)', 'China, Hong Kong SAR','Countries with Access to the Sea',       'Countries with Access to the Sea: Africa',       'Countries with Access to the Sea: Asia',       'Countries with Access to the Sea: Europe',       'Countries with Access to the Sea: Latin America and the Caribbean',       'Countries with Access to the Sea: Northern America',       'Countries with Access to the Sea: Oceania',       'China, Macao SAR', 'China, Taiwan Province of China','Commonwealth of Independent States (CIS)',       'Commonwealth of Nations', 'Commonwealth: Africa',       'Commonwealth: Asia', 'Commonwealth: Caribbean and Americas',       'Commonwealth: Europe', 'Commonwealth: Pacific','Countries with Access to the Sea',       'Countries with Access to the Sea: Africa',       'Countries with Access to the Sea: Asia',       'Countries with Access to the Sea: Europe',       'Countries with Access to the Sea: Latin America and the Caribbean',       'Countries with Access to the Sea: Northern America',       'Countries with Access to the Sea: Oceania','Denmark (and dependencies)','ECE: North America-2', 'ECE: UNECE-52',       'ECLAC: Latin America', 'ECLAC: The Caribbean',       'ESCAP region: East and North-East Asia',       'ESCAP region: North and Central Asia', 'ESCAP region: Pacific',       'ESCAP region: South and South-West Asia',       'ESCAP region: South-East Asia',       'ESCAP: ADB Developing member countries (DMCs)',       'ESCAP: ADB Group A (Concessional assistance\xa0only)',       'ESCAP: ADB Group B\xa0(OCR blend)',       'ESCAP: ADB Group C (Regular OCR only)', 'ESCAP: ASEAN',       'ESCAP: Central Asia', 'ESCAP: ECO', 'ESCAP: HDI groups',       'ESCAP: Landlocked countries (LLDCs)',       'ESCAP: Least Developed Countries (LDCs)',       'ESCAP: Pacific island dev. econ.', 'ESCAP: SAARC',       'ESCAP: WB High income econ.', 'ESCAP: WB Low income econ.',       'ESCAP: WB Lower middle income econ.',       'ESCAP: WB Upper middle income econ.', 'ESCAP: WB income groups',       'ESCAP: high HDI', 'ESCAP: high income', 'ESCAP: income groups',       'ESCAP: low HDI', 'ESCAP: low income', 'ESCAP: lower middle HDI',       'ESCAP: lower middle income',       'ESCAP: other Asia-Pacific countries/areas',       'ESCAP: upper middle HDI', 'ESCAP: upper middle income',       'ESCWA: Arab countries', 'ESCWA: Arab least developed countries',       'ESCWA: Gulf Cooperation Council countries',       'ESCWA: Maghreb countries', 'ESCWA: Mashreq countries',       'ESCWA: member countries', 'Early-demographic dividend',       'East African Community (EAC)', 'East Asia', 'East Asia & Pacific',       'East Asia & Pacific (IDA & IBRD)',       'East Asia & Pacific (excluding high income)', 'Eastern Africa',       'Eastern Asia', 'Eastern Europe', 'Eastern European Group',       'Eastern Sub-Saharan Africa', 'Eastern and South-Eastern Asia',       'Economic Community of Central African States (ECCAS)',       'Economic Community of West African States (ECOWAS)',       'Economic Cooperation Organization (ECO)','Eurasian Economic Community (Eurasec)', 'Euro area',       'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD)',       'Europe & Central Asia (excluding high income)', 'Europe (48)',       'Europe and Northern America', 'European Community (EC: 12)',       'European Free Trade Agreement (EFTA)', 'European Union',       'European Union (EU: 15)', 'European Union (EU: 28)','Greater Arab Free Trade Area (GAFTA)','Group of 77 (G77)', 'Group of Eight (G8)',       'Group of Seven (G7)', 'Group of Twenty (G20) - member states','Gulf Cooperation Council (GCC)',       'Heavily indebted poor countries (HIPC)', 'High SDI',       'High income', 'High-income', 'High-income Asia Pacific',       'High-income countries', 'High-middle SDI',       'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only',       'IDA total', 'LLDC: Africa',       'LLDC: Asia', 'LLDC: Europe', 'LLDC: Latin America',       'Land-locked Countries', 'Land-locked Countries (Others)',       'Land-locked Developing Countries (LLDC)','Late-demographic dividend', 'Latin America & Caribbean',       'Latin America & Caribbean (IDA & IBRD)',       'Latin America & Caribbean (excluding high income)',       'Latin America and Caribbean', 'Latin America and the Caribbean',       'Latin American Integration Association (ALADI)',       'Latin American and Caribbean Group (GRULAC)',       'League of Arab States (LAS, informal name: Arab League)',       'Least developed countries',       'Least developed countries: UN classification',       'Least developed: Africa', 'Least developed: Asia',       'Least developed: Latin America and the Caribbean',       'Least developed: Oceania', 'Less developed regions',       'Less developed regions, excluding China',       'Less developed regions, excluding least developed countries',       'Less developed: Africa', 'Less developed: Asia',       'Less developed: Latin America and the Caribbean',       'Less developed: Oceania','Low & middle income', 'Low SDI', 'Low income',       'Low-income countries', 'Low-middle SDI', 'Lower middle income',       'Lower-middle-income countries','Middle Africa',       'Middle East & North Africa',       'Middle East & North Africa (IDA & IBRD)',       'Middle East & North Africa (excluding high income)', 'Middle SDI',       'Middle income', 'Middle-income countries', 'More developed regions', 'More developed: Asia',       'More developed: Europe', 'More developed: Northern America',
       'More developed: Oceania','New EU member states (joined since 2004)','No income group available', 'Non-Self-Governing Territories','North Africa and Middle East', 'North America',       'North American Free Trade Agreement (NAFTA)',       'North Atlantic Treaty Organization (NATO)', 'Northern Africa',       'Northern Africa and Western Asia', 'Northern America',       'Northern Europe','OECD members', 'Oceania',       'Oceania (excluding Australia and New Zealand)',       'Organisation for Economic Co-operation and Development (OECD)',       'Organization for Security and Co-operation in Europe (OSCE)',       'Organization of American States (OAS)',       'Organization of Petroleum Exporting countries (OPEC)',       'Organization of the Islamic Conference (OIC)',       'Other small states', 'Pacific island small states','Polynesia',
       'Post-demographic dividend', 'Pre-demographic dividend','SIDS Atlantic, and Indian Ocean, Mediterranean and South China Sea (AIMS)',
       'SIDS Caribbean', 'SIDS Pacific','Shanghai Cooperation Organization (SCO)','Small Island Developing States (SIDS)', 'Small states',       'South America', 'South Asia', 'South Asia (IDA & IBRD)',       'South Asian Association for Regional Cooperation (SAARC)','South-Eastern Asia',       'Southeast Asia', 'Southeast Asia, East Asia, and Oceania',       'Southern Africa', 'Southern African Development Community (SADC)',       'Southern Asia', 'Southern Common Market (MERCOSUR)',       'Southern Europe', 'Southern Latin America',       'Southern Sub-Saharan Africa',       'Sub-Saharan Africa', 'Sub-Saharan Africa (IDA & IBRD)',       'Sub-Saharan Africa (excluding high income)','Tropical Latin America','UN-ECE: member countries', 'UNFPA Regions',
       'UNFPA: Arab States (AS)', 'UNFPA: Asia and the Pacific (AP)',
       'UNFPA: East and Southern Africa (ESA)',
       'UNFPA: Eastern Europe and Central Asia (EECA)',
       'UNFPA: Latin America and the Caribbean (LAC)',
       'UNFPA: West and Central Africa (WCA)', 'UNICEF PROGRAMME REGIONS',
       'UNICEF Programme Regions: East Asia and Pacific (EAPRO)',
       'UNICEF Programme Regions: Eastern Caribbean',
       'UNICEF Programme Regions: Eastern and Southern Africa (ESARO)',
       'UNICEF Programme Regions: Europe and Central Asia (CEECIS)',
       'UNICEF Programme Regions: Latin America',
       'UNICEF Programme Regions: Latin America and Caribbean (LACRO)',
       'UNICEF Programme Regions: Middle East and North Africa (MENARO)',
       'UNICEF Programme Regions: South Asia (ROSA)',
       'UNICEF Programme Regions: West and Central Africa (WCARO)',
       'UNICEF REGIONS', 'UNICEF Regions: East Asia and Pacific',
       'UNICEF Regions: Eastern Europe and Central Asia',
       'UNICEF Regions: Eastern and Southern Africa',       'UNICEF Regions: Europe and Central Asia',       'UNICEF Regions: Latin America and Caribbean',       'UNICEF Regions: Middle East and North Africa',       'UNICEF Regions: North America', 'UNICEF Regions: South Asia',       'UNICEF Regions: Sub-Saharan Africa',       'UNICEF Regions: West and Central Africa',       'UNICEF Regions: Western Europe',
       'UNITED NATIONS Regional Groups of Member States','United Nations Economic Commission for Africa (UN-ECA)',
       'United Nations Economic Commission for Latin America and the Caribbean (UN-ECLAC)',
       'United Nations Economic and Social Commission for Asia and the Pacific (UN-ESCAP) Regions',
       'United Nations Member States','Upper middle income', 'Upper-middle-income countries','Upper middle income', 'Upper-middle-income countries','WB region: East Asia and Pacific (excluding high income)',
       'WB region: Europe and Central Asia (excluding high income)',
       'WB region: Latin America and Caribbean (excluding high income)',
       'WB region: Middle East and North Africa (excluding high income)',
       'WB region: South Asia (excluding high income)',
       'WB region: Sub-Saharan Africa (excluding high income)',
       'WHO Regions', 'WHO: African region (AFRO)',
       'WHO: Americas (AMRO)', 'WHO: Eastern Mediterranean Region (EMRO)',
       'WHO: European Region (EURO)',
       'WHO: South-East Asia region (SEARO)',
       'WHO: Western Pacific region (WPRO)', 'Wales',
       'West African Economic and Monetary Union (UEMOA)',
       'Western Africa', 'Western Asia', 'Western Europe',
       'Western European and Others Group (WEOG)', 'Western Sahara',
       'Western Sub-Saharan Africa',
       'World Bank Regional Groups (developing only)','Fragile and conflict affected situations','France (and dependencies)','Netherlands (and dependencies)',
       'New Caledonia', 'New Zealand (and dependencies)' ]

df_demo_sorted = df_demo.copy()

# Delete these rows

for cnt in countries_drop:
    df_demo_sorted.drop(df_demo_sorted.index[df_demo_sorted.Country==cnt],inplace=True,axis='index')

In [191]:
df_demo_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265 entries, 0 to 555
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              265 non-null    object 
 1   Cardio Death Rate    198 non-null    float64
 2   Diabetes Percentage  206 non-null    float64
 3   Obesity              167 non-null    float64
 4   Undernourished       162 non-null    float64
 5   PopMale              200 non-null    float64
 6   PopFemale            200 non-null    float64
 7   PopTotal             200 non-null    float64
 8   Total Population     200 non-null    float64
dtypes: float64(8), object(1)
memory usage: 20.7+ KB


In [192]:
# Combine rows of same country but under different name

dic = {'Curacao': ['Curaçao'],'Czechia': ['Czech Republic'], 'Congo':['Democratic Republic of Congo','Democratic Republic of the Congo'],
    'Korea, South':["Dem. People's Republic of Korea",'South Korea'],'Korea, North':['North Korea', 'Republic of Korea'],
    'Laos':["Lao People's Democratic Republic"], 'Micronesia': ['Micronesia (Fed. States of)','Micronesia (country)'],
    'Reunion':['Réunion'],'Moldova':['Republic of Moldova'],'Syria':['Syrian Arab Republic'],'Taiwan*':['Taiwan'],
    'Timor-Leste':['Timor'],'Tanzania':['United Republic of Tanzania'],'Venezuela':['Venezuela (Bolivarian Republic of)'],
    'Vietnam':['Viet Nam'],'Brunei':['Brunei Darussalam'],'Cabo Verde':['Cape Verde'],"Cote d'Ivoire":["Côte d'Ivoire"],
       'Iran': ['Iran (Islamic Republic of)'],'Russia': ['Russian Federation'],'Palestine':['State of Palestine'],'North Macedonia':['Macedonia']}
for key in dic.keys():
    bool_ctr = df_demo_sorted.Country==key
    ctr_int = int(list(df_demo_sorted.index[bool_ctr])[0])
    for col in df_demo_sorted.columns[1:]:
        if list(np.isnan(df_demo_sorted[col].loc[bool_ctr]))[0]:
            for ctr2 in dic[key]:
                if ~list(np.isnan(df_demo_sorted[col].loc[df_demo_sorted.Country==ctr2]))[0]:
                    df_demo_sorted.at[ctr_int,col] = df_demo_sorted[col].loc[df_demo_sorted.Country==ctr2]
                    continue
    # Delete rows of secondary entry
    for ctr2 in dic[key]:
        df_demo_sorted.drop(df_demo_sorted.index[df_demo_sorted.Country==ctr2],inplace=True,axis='index') 

In [193]:
df_demo_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 0 to 474
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              239 non-null    object 
 1   Cardio Death Rate    196 non-null    float64
 2   Diabetes Percentage  204 non-null    float64
 3   Obesity              167 non-null    float64
 4   Undernourished       162 non-null    float64
 5   PopMale              197 non-null    float64
 6   PopFemale            197 non-null    float64
 7   PopTotal             197 non-null    float64
 8   Total Population     197 non-null    float64
dtypes: float64(8), object(1)
memory usage: 18.7+ KB


In [195]:
bool_nan = np.isnan(df_demo_sorted['Cardio Death Rate']) | np.isnan(df_demo_sorted['Diabetes Percentage']) | np.isnan(df_demo_sorted['Undernourished'])|np.isnan(df_demo_sorted['PopMale'])
np.sort(df_demo_sorted.Country[bool_nan])


array(['Africa', 'American Samoa', 'Andorra', 'Anguilla',
       'Antigua and Barbuda', 'Aruba', 'Asia', 'Bahamas', 'Bahrain',
       'Bermuda', 'Bhutan', 'Bolivia', 'British Virgin Islands', 'Brunei',
       'Burundi', 'Cayman Islands', 'Christmas Island', 'Cocos Islands',
       'Comoros', 'Cook Islands', 'Curacao', 'Dominica',
       'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Europe',
       'Faeroe Islands', 'French Guiana', 'French Polynesia', 'Gibraltar',
       'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guernsey',
       'Hong Kong', 'Isle of Man', 'Jersey', 'Korea, North',
       'Korea, South', 'Kosovo', 'Libya', 'Liechtenstein', 'Macao',
       'Marshall Islands', 'Martinique', 'Mayotte', 'Melanesia',
       'Micronesia', 'Moldova', 'Monaco', 'Montserrat', 'Nauru', 'Niue',
       'Norfolk Island', 'Northern Ireland', 'Northern Mariana Islands',
       'Palau', 'Palestine', 'Papua New Guinea', 'Pitcairn',
       'Puerto Rico', 'Qatar', 'Reunion', 'Saint Helena',
       '

## Merge COVID-19 data frame with demographics

In [202]:
df_base_sorted = pd.merge(left=df_base,right=df_demo_sorted,how='left',on='Country')
df_base_sorted = df_base_sorted.dropna(inplace=True)

In [203]:
df_base_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 186
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              187 non-null    object 
 1   Confirmed            187 non-null    int64  
 2   Deaths               187 non-null    int64  
 3   Recovered            187 non-null    int64  
 4   Active               187 non-null    int64  
 5   Cardio Death Rate    173 non-null    float64
 6   Diabetes Percentage  176 non-null    float64
 7   Obesity              158 non-null    float64
 8   Undernourished       152 non-null    float64
 9   PopMale              169 non-null    float64
 10  PopFemale            169 non-null    float64
 11  PopTotal             169 non-null    float64
 12  Total Population     169 non-null    float64
dtypes: float64(8), int64(4), object(1)
memory usage: 20.5+ KB


## Store CSV files

In [209]:
df_base_sorted.to_csv('../data/interim/COVID_base.csv')
df_demo_sorted.to_csv('../data/interim/Demographics_base.csv')

### NOT NEEDED ANYMORE

### Population data

In [5]:
population_link = "https://datacenter.prb.org/download/international/indicator/population/csv"

In [6]:
import requests
import csv

myfile = requests.get(population_link,allow_redirects=True)
myfile_content = myfile.content.decode('utf-8')
cr = csv.reader(myfile_content.splitlines(),delimiter=',')
my_list = list(cr)
popfile = csv.writer(open(raw_data+'population.csv','wt'))
for row in my_list[4:]:
    popfile.writerow(row)
df_pop = pd.read_csv(raw_data+'population.csv')
df_pop.head()    


,FIPS,Name,Type,TimeFrame,Data
0,WORLD,WORLD,World,2019,7691.463
1,AFRICA,AFRICA,Sub-Region,2019,1305.215
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,239.895
3,DZ,Algeria,Country,2019,43.406
4,EG,Egypt,Country,2019,99.064
